In [25]:
from ase.db import connect
from clease.tools import update_db
from ase.io import read
import clease
from clease import Evaluate
import clease.plot_post_process as pp
import matplotlib.pyplot as plt
from clease.settings import Concentration
from clease.settings import CEBulk
import numpy as np

# Set the Concentration Limits

In [6]:
basis_elements = [['V', 'Cr', 'Ti', 'W', 'Zr']]

# Equality constraint: concentrations sum up to 1
A_eq = [[1.0, 1.0, 1.0, 1.0, 1.0]]
b_eq = [1.0]

# Lower bound constraints
# At least 0.8 of V, and at least 0.01 of each of the others
A_lb = [[1, 0, 0, 0, 0],  # for V
        [0, 1, 0, 0, 0],  # for Cr
        [0, 0, 1, 0, 0],  # for Ti
        [0, 0, 0, 1, 0],  # for W
        [0, 0, 0, 0, 1]]  # for Zr
b_lb = [0.8, 0.01, 0.01, 0.01, 0.01]

# Upper bound constraints not directly supported in CLEASE setup,
# you can enforce these when sampling/creating configurations.

In [11]:
v_min = 0.8
v_max = 1.0
cr_min = 0.01
cr_max = 0.05
ti_min = 0.01
ti_max = 0.05
w_min = 0.01
w_max = 0.05
zr_min = 0.01
zr_max = 0.05
conc_ranges = [(v_min,v_max), (cr_min,cr_max), (ti_min,ti_max), (w_min,w_max), (zr_min,zr_max)]

In [12]:
#conc = Concentration(basis_elements=basis_elements, A_lb=A_lb, b_lb=b_lb, A_eq=A_eq, b_eq=b_eq)
conc = Concentration(basis_elements=basis_elements)
conc.set_conc_ranges(ranges=[conc_ranges])


# Create settings for CE

In [36]:
settings = CEBulk(crystalstructure='bcc',
                  a=3.01,
                  size=[4,4,4],
                  supercell_factor = 64,
                  concentration=conc,
                  db_name = '/Users/myless/Packages/structure_maker/CLEASE_DBs/VCrTiWZr_new.db',
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [15]:
vcrtiwzr_db = connect('/Users/myless/Packages/structure_maker/CLEASE_DBs/VCrTiWZr.db')

In [27]:
new_vcrtiwzr_db = connect('/Users/myless/Packages/structure_maker/CLEASE_DBs/VCrTiWZr_new.db')
for row in vcrtiwzr_db.select():
    atoms = row.toatoms()
    new_vcrtiwzr_db.write(atoms,converged=True)

# Setup the CE Model

In [41]:
from clease import Evaluate
import clease.plot_post_process as pp 
import matplotlib.pyplot as plt
from clease.tools import reconfigure
from clease.corr_func import CorrFunction

corr_fun = CorrFunction(settings=settings)
select_cond = [('converged', '=', True)]
eva = Evaluate(settings=settings, scoring_scheme='k-fold',nsplits=10,select_cond=select_cond)



ValueError: Requested template violates the constraints!